In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#2. specify parameters
pipeline_params={
}
step_params={
}
substep_params={   
}

In [ ]:
#3 define substep interface
from sinara.substep import NotebookSubstep, default_param_values, ENV_NAME, PIPELINE_NAME, ZONE_NAME, STEP_NAME, RUN_ID, ENTITY_NAME, ENTITY_PATH, SUBSTEP_NAME

substep = NotebookSubstep(pipeline_params, step_params, substep_params, **default_param_values("params/step_params.json"))

substep.interface(
    inputs =
    [ { STEP_NAME: "1_data_import", ENTITY_NAME: "train_coco_data",  RUN_ID: "run-23-10-12-065624"},
      { STEP_NAME: "1_data_import", ENTITY_NAME: "eval_coco_data",  RUN_ID: "run-23-10-12-065624"},
      { STEP_NAME: "1_data_import", ENTITY_NAME: "cache_config",  RUN_ID: "run-23-10-12-065624"}
    ],
    
    outputs = 
    [
        { ENTITY_NAME: "aug_dataset"},
        { ENTITY_NAME: "config"}
    ],
    
    tmp_outputs =
    [
        { ENTITY_NAME: "cache_data" }
    ]
)

substep.print_interface_info()

substep.exit_in_visualize_mode()

In [ ]:
#4 get substep.interface
inputs = substep.inputs(step_name = "1_data_import")
outputs = substep.outputs()
tmp_outputs = substep.tmp_outputs()

print(f"{inputs.train_coco_data=}")
print(f"{inputs.eval_coco_data=}")
print(f"{inputs.cache_config=}")

print(f"{outputs.aug_dataset=}")
print(f"{outputs.config=}")

print(f"{tmp_outputs.cache_data=}")

In [ ]:
#5 run spark
from sinara.spark import SinaraSpark

spark = SinaraSpark.run_session(0)
SinaraSpark.ui_url()

In [ ]:
import os.path as osp
import os

In [ ]:
os.makedirs(tmp_outputs.cache_data, exist_ok=True)
os.makedirs(outputs.aug_dataset, exist_ok=True)
os.makedirs(outputs.config, exist_ok=True)

In [ ]:
import json

config_fn = os.path.join(inputs.cache_config, 'config.json')

with open(config_fn) as f_id:
    CONFIG = json.load(f_id)

CONFIG

#### Загрузка обработанных датасетов (пред. компонента 1_data_import)

In [ ]:
def save_file(row): 
    total_dir_name = outputs.aug_dataset
    total_img_path = osp.join(total_dir_name, row.file_names)
    os.makedirs(total_dir_name, exist_ok=True)
    with open(total_img_path, 'wb') as f_id:
        f_id.write(row.files_binary)


In [ ]:
%%time

# CACHE TEST

print(f"spark read start")

df_spark = spark.read.parquet(inputs.eval_coco_data)
df_spark.foreach(save_file)

In [ ]:
%%time

# CACHE TRAIN

print(f"spark read start")

df_spark = spark.read.parquet(inputs.train_coco_data)
df_spark.foreach(save_file)

In [ ]:
CONFIG

In [ ]:
import json

train_coco_annotation = os.path.join(inputs.cache_config, CONFIG["train_coco_annotation"])
val_coco_annotation = os.path.join(inputs.cache_config, CONFIG["val_coco_annotation"])

with open(train_coco_annotation) as f_id:
    train_coco = json.load(f_id)

with open(val_coco_annotation) as f_id:
    val_coco = json.load(f_id)

with open(osp.join(outputs.config,  CONFIG["train_coco_annotation"]), 'w') as f:
    json.dump(train_coco, f, indent=4)

with open(osp.join(outputs.config,  CONFIG["val_coco_annotation"]), 'w') as f:
    json.dump(val_coco, f, indent=4)
    
config_path = osp.join(outputs.config, "config.json")
with open(config_path, 'w') as f:
    json.dump(CONFIG, f, indent=4)

In [ ]:
!cat {config_path}

In [ ]:
#9 stop spark
SinaraSpark.stop_session()